In [3]:
import copy
import math

import nbimporter
import numpy

import nu # 方位係数
import f  # 日射取得率補正係数
import H  # 温度差係数

Importing Jupyter notebook from nu.ipynb
Importing Jupyter notebook from f.ipynb
Importing Jupyter notebook from H.ipynb


# UA値・ηA値より部位のU値・η値を決定する

## 1. 概要

UA値・ηA値より部位のU値・η値を決定する方法の主な考え方は、以下のとおり。  

・外皮について、極力、与えられたUA値、ηA値を満たすように、基準値計算用の住宅のU値（仕様規定の部位別U値に比例）と窓のη値を決定する。    
・非現実的な設定は許容しない（例：窓の日射熱取得率が0未満）。

## 2. 部位の性能の設定

In [7]:
class Parts:
    
    def __init__(self, area, ptype, nextspace, direction):
        self.__area = area
        self.__ptype = ptype
        self.__nextspace = nextspace
        self.__direction = direction
    
    @property
    def area(self):
        return self.__area

    @property
    def ptype(self):
        return self.__ptype
     
    @property
    def nextspace(self):
        return self.__nextspace  
    
    @property
    def direction(self):
        return self.__direction

### 2.1 各種設定値の取得

#### 1) 総外皮面積の算出

$$
\displaystyle
A_{total}=  \sum_{i}{A_i}
$$

$A_i$: 第$i$番目の外皮の部位の面積, m<sup>2</sup>  
$A_{total}$: 総外皮面積, m<sup>2</sup>  

#### Function

In [8]:
def get_total_area( d ):
    
    d_area = []
    for x in ['general_parts','windows','doors','earthfloors'] :
        if (x in d) == True :
            d_area.extend(d[x])

    return sum(x['area'] for x in d_area)

#### Example

In [9]:
d={
    'general_parts'        : [{'area': 1}, {'area': 2}],
    'windows'              : [{'area': 3}],
    'doors'                : [{'area': 4}],
    'earthfloors'          : [{'area': 5}],
    'earthfloor_perimeters': [{'area': 6}]
}

get_total_area( d )

15

#### 2) 部位種類の設定

#### 全ての部位

In [10]:
def get_list_part_type():
    return ['roof','ceiling','wall','floor','boundary_ceiling','boundary_wall','boundary_floor','window','door','earthfloor_perimeter']

#### 日射の当たる部位

In [11]:
def get_list_sunshine_part_type():
    return ['roof','ceiling','wall','window','door']

#### 日射の当たらない部位

In [12]:
def get_list_no_sunshine_part_type():
    return ['floor','boundary_ceiling','boundary_wall','boundary_floor','earthfloor_perimeter']

#### 3) 部位別の熱貫流率の基準値、想定値

基準値計算用の部位別の熱貫流率の基準値または想定値は、現行の省エネルギー基準等を参考に設定する。

　屋根または天井・壁・床・開口部・土間床等の外周部：  
 　　住宅に係るエネルギーの使用の合理化に関する設計、施工及び維持保全の指針の一部を改正する告示（平成28年4月1日施行）  
　壁・床（8地域）：RC150mm（無断熱）を想定  
　土間床等の外周部（8地域）：土間床等の外周部及び基礎等の線熱貫流率の入力を行わない場合の値   
　開口部（8地域）：単板ガラスの熱貫流率を想定

部位別の熱貫流率の基準値、想定値, W/(㎡・K)

| 部位 | 　 | 1地域 | 2地域 | 3地域 | 4地域 | 5地域 | 6地域 | 7地域 | 8地域 |
|---|---|---|---|---|---|---|---|---|---|
| 屋根・天井 | 　 | 0.17 | 0.17 | 0.24 | 0.24 | 0.24 | 0.24 | 0.24 | 0.24 |
| 壁 | 　 | 0.35 | 0.35 | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 4.10 |
| 床 | 外気に接する部分 | 0.24 | 0.24 | 0.24 | 0.34 | 0.34 | 0.34 | 0.34 | 3.52 |
| 床 | その他の部分 | 0.34 | 0.34 | 0.34 | 0.48 | 0.48 | 0.48 | 0.48 | 2.54 |
| 開口部 | 　 | 2.33 | 2.33 | 3.49 | 4.65 | 4.65 | 4.65 | 4.65 | 6.51 |
| 土間床等の外周部 | 外気に接する部分 | 0.37 | 0.37 | 0.37 | 0.53 | 0.53 | 0.53 | 0.53 | 1.8 |
| 土間床等の外周部 | その他の部分 | 0.53 | 0.53 | 0.53 | 0.76 | 0.76 | 0.76 | 0.76 | 1.8 |

#### Function

In [13]:
def get_std_U(region, part_type) :

    return {
        'roof'                 : {1: 0.17, 2: 0.17, 3: 0.24, 4: 0.24, 5: 0.24, 6: 0.24, 7: 0.24, 8: 0.24 },
        'ceiling'              : {1: 0.17, 2: 0.17, 3: 0.24, 4: 0.24, 5: 0.24, 6: 0.24, 7: 0.24, 8: 0.24 },
        'wall'                 : {1: 0.35, 2: 0.35, 3: 0.53, 4: 0.53, 5: 0.53, 6: 0.53, 7: 0.53, 8: 4.10 },
        'floor'                : {1: 0.34, 2: 0.34, 3: 0.34, 4: 0.48, 5: 0.48, 6: 0.48, 7: 0.48, 8: 3.52 },
        'boundary_ceiling'     : {1: 0.17, 2: 0.17, 3: 0.24, 4: 0.24, 5: 0.24, 6: 0.24, 7: 0.24, 8: 0.24 },
        'boundary_wall'        : {1: 0.35, 2: 0.35, 3: 0.53, 4: 0.53, 5: 0.53, 6: 0.53, 7: 0.53, 8: 4.10 },
        'boundary_floor'       : {1: 0.34, 2: 0.34, 3: 0.34, 4: 0.48, 5: 0.48, 6: 0.48, 7: 0.48, 8: 3.52 },
        'window'               : {1: 2.33, 2: 2.33, 3: 3.49, 4: 4.65, 5: 4.65, 6: 4.65, 7: 4.65, 8: 6.51 },
        'door'                 : {1: 2.33, 2: 2.33, 3: 3.49, 4: 4.65, 5: 4.65, 6: 4.65, 7: 4.65, 8: 6.51 },
        'earthfloor_perimeter' : {1: 0.53, 2: 0.53, 3: 0.53, 4: 0.76, 5: 0.76, 6: 0.76, 7: 0.76, 8: 1.80 }
    }[part_type][region]

#### Example

In [14]:
get_std_U(6, 'roof')

0.24

#### 4) 部位別の熱貫流率の上下限値

下限値は十分に小さい値、上限値は合板12mm、土間床等の外周部の上限値は土間床等の外周部及び基礎等の線熱貫流率の入力を行わない場合の値を想定。

| 　 | 下限値[W/(㎡・K)] | 上限値[W/(㎡・K)] |
|---|---|---|
| 屋根 | 0.001 | 1/(0.09+0.0095/0.22+0.04) = 5.77 |
| 天井 | 0.001 | 1/(0.09+0.0095/0.22+0.09) = 4.48 |
| 壁 | 0.001 | 1/(0.11+0.0095/0.22+0.04) = 5.18 |
| 床 | 0.001 | 1/(0.15+0.012/0.16+0.04) = 3.77 |
| 界床（天井） | 0.001 | 1/(0.09+0.0095/0.22+0.09) = 4.48 |
| 界壁 | 0.001 | 1/(0.11+0.0095/0.22+0.11) = 3.80 |
| 界床 | 0.001 | 1/(0.15+0.012/0.16+0.15) = 2.67 |
| 開口部 | 0.001 | 6.51 |
| 土間床等の外周部 | 0.001 | 1.8 |

#### Function

In [18]:
def get_min_U(ptype):    # 熱貫流率の下限値 W/m2 K
    return {
        'roof'            : 0.001,
        'ceiling'         : 0.001,
        'wall'            : 0.001,
        'floor'           : 0.001,
        'boundary_ceiling': 0.001,
        'boundary_wall'   : 0.001,
        'boundary_floor'  : 0.001,
        'window'          : 0.001,
        'door'            : 0.001,
        'earthfloor_perimeter'     : 0.001
    }[ptype]

In [19]:
def get_max_U(ptype):    # 部位別の熱貫流率の上限値, W/m2 K
    return {
        'roof'            : 1/(0.09+0.0095/0.22+0.04),
        'ceiling'         : 1/(0.09+0.0095/0.22+0.09),
        'wall'            : 1/(0.11+0.0095/0.22+0.04),
        'floor'           : 1/(0.15+0.012 /0.16+0.04),
        'boundary_ceiling': 1/(0.09+0.0095/0.22+0.09),
        'boundary_wall'   : 1/(0.11+0.0095/0.22+0.11),
        'boundary_floor'  : 1/(0.15+0.012 /0.16+0.15),
        'window'          : 6.51,
        'door'            : 6.51,
        'earthfloor_perimeter'     : 100
    }[ptype]

### 3.2 単位温度差当たりの外皮熱損失量

単位温度差当たりの外皮熱損失量とは、部位の面積・長さと熱貫流率と温度差係数の積であり、単位は、W/K である。

#### 1) 全外皮の部位

$$
\displaystyle
q_{std,U} = \sum_{i}(A_{i} \times U_{std,i} \times H_{i}) +  \sum_{i}(L_{i} \times Ψ_{std,i} \times H_{i})
$$

$q_{std,U}$: 基準U値に基づく全部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$U_{std,i}$: 第$i$番目の外皮の部位の熱貫流率の基準値, W/m<sup>2</sup> K  
$Ψ_{std,i}$: 第$i$番目の外皮の部位の線熱貫流率の基準値, W/mK  
$H_i$: 第$i$番目の外皮の部位の温度差係数, -  

#### 2) 日射が当たる外皮の部位

$$
\displaystyle
q_{s,std,U} = \sum_i(A_{s,i} \times U_{s,std,i} \times H_{s,i})
$$

$q_{s,std,U}$: 基準U値に基づく日射が当たる外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$A_{s,i}$: 第$i$番目の日射が当たる外皮の部位の面積, m<sup>2</sup>  
$U_{s,std,i}$: 第$i$番目の日射が当たる外皮の部位の熱貫流率の基準値, W/m<sup>2</sup> K  
$H_{s,i}$: 第$i$番目の日射が当たる外皮の部位の温度差係数, -  

#### 3) 日射が当たらない外皮の部位

$$
\displaystyle
q_{n,std,U} = \sum(A_{n,i} \times U_{n,std,i} \times H_{n,i}) +  \sum(L_{n,i} \times Ψ_{n,std,i} \times H_{n,i})
$$

$$
\displaystyle
q_{n,max,U} = \sum(A_{n,i} \times U_{n,max,i} \times H_{n,i}) + \sum(L_{n,i} \times Ψ_{n,max,i} \times H_{n,i})
$$

$$
\displaystyle
q_{n,min,U} = \sum(A_{n,i} \times U_{n,min,i} \times H_{n,i}) + \sum(L_{n,i} \times Ψ_{n,min,i} \times H_{n,i})
$$

$q_{n,std,U}$: 基準U値に基づく日射が当たらない外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$q_{n,max,U}$: 最大U値に基づく日射が当たらない外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$q_{n,min,U}$: 最小U値に基づく日射が当たらない外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$A_{n,i}$: 第$i$番目の日射が当たらない外皮の部位の面積, m<sup>2</sup>  
$L_{n,i}$: 第$i$番目の日射が当たらない外皮の部位の長さ, m</sup>  
$U_{n,std,i}$: 第$i$番目の日射が当たらない外皮の部位の熱貫流率の基準値, W/m<sup>2</sup> K  
$U_{n,max,i}$: 第$i$番目の日射が当たらない外皮の部位の熱貫流率の最大値, W/m<sup>2</sup> K  
$U_{n,min,i}$: 第$i$番目の日射が当たらない外皮の部位の熱貫流率の最小値, W/m<sup>2</sup> K  
$Ψ_{n,std,i}$: 第$i$番目の日射が当たらない外皮の部位の線熱貫流率の基準値, W/m</sup> K  
$Ψ_{n,max,i}$: 第$i$番目の日射が当たらない外皮の部位の線熱貫流率の最大値, W/m</sup> K  
$Ψ_{n,min,i}$: 第$i$番目の日射が当たらない外皮の部位の線熱貫流率の最小値, W/m</sup> K  
$H_{n,i}$: 第$i$番目の日射が当たらない外皮の部位の温度差係数, - 

#### Function

In [20]:
def get_q_std_U(region, parts, l_target):
    
    return sum((part.area * get_std_U(region, part.ptype) * H.get_H_by_nextspace(region, part.nextspace) if part.ptype in l_target else 0)
               for part in parts)

In [21]:
def get_q_max_U(region, parts, l_target):
    
    return sum((part.area * get_max_U(part.ptype) * H.get_H_by_nextspace(region, part.nextspace) if part.ptype in l_target else 0)
               for part in parts)

In [22]:
def get_q_min_U(region, parts, l_target):
    
    return sum((part.area * get_min_U(part.ptype) * H.get_H_by_nextspace(region, part.nextspace) if part.ptype in l_target else 0)
               for part in parts)

### 3.3 熱貫流率の調整係数の最大値・最小値

#### 1) 日射が当たる外皮の部位

$$
\displaystyle
f_{U,s,max}= \min \left(\frac{ U_{A,std} \times A_{total} - q_{n,min,U} }{ q_{s,std,U} },\min \left(\frac{U_{s,max,i}}{U_{s,std,i}} \right) \right)
$$

$$
\displaystyle
f_{U,s,min}= \max \left(\frac{ U_{A,std} \times A_{total} - q_{n,max,U} }{ q_{s,std,U} },\max \left(\frac{U_{s,min,i}}{U_{s,std,i}} \right) \right)
$$

$f_{U,s,max}$: 日射が当たる外皮の部位の熱貫流率の調整係数の最大値, -  
$f_{U,s,min}$: 日射が当たる外皮の部位の熱貫流率の調整係数の最小値, -  
$U_{A,std}$: 外皮平均熱貫流率の基準値, W/m<sup>2</sup> K  
$q_{s,std,U}$: 基準U値に基づく日射が当たる外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$q_{n,max,U}$: 最大U値に基づく日射が当たらない外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$q_{n,min,U}$: 最小U値に基づく日射が当たらない外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$U_{s,std,i}$: 第$i$番目の日射が当たる外皮の部位の熱貫流率の基準値, W/m<sup>2</sup> K  
$U_{s,max,i}$: 第$i$番目の日射が当たる外皮の部位の熱貫流率の最大値, W/m<sup>2</sup> K  
$U_{s,min,i}$: 第$i$番目の日射が当たる外皮の部位の熱貫流率の最小値, W/m<sup>2</sup> K  

In [23]:
def get_fUs_range(region, std_UA, total_area, qs_std_U, qn_max_U, qn_min_U, l_sunshine_part_type):

    # 日射の当たる部位の熱貫流率補正係数の最小値
    fUs_max = min(
        (std_UA * total_area - qn_min_U) / qs_std_U,
        min(get_max_U(x) / get_std_U(region, x) for x in l_sunshine_part_type)
    )
    
    # 日射の当たる部位の熱貫流率補正係数の最大値
    fUs_min = max(
        (std_UA * total_area - qn_max_U) / qs_std_U,
        max(get_min_U(x) / get_std_U(region, x) for x in l_sunshine_part_type)
    )
    
    return fUs_max, fUs_min

#### 2) 日射が当たらない外皮の部位（床、土間床）

$$
\displaystyle
f_{U,n,max}= \min \left(\frac{ U_{A,std} \times A_{total} - q_{s,std,U} \times \max(\frac{U_{s,min,i}}{U_{s,std,i}}) }{ q_{n,std,U} },\frac{U_{n,max,i}}{U_{n,std,i}} \right)
$$

$$
\displaystyle
f_{U,n,min}= \max \left(\frac{ U_{A,std} \times A_{total} - q_{s,std,U} \times \min(\frac{U_{s,max,i}}{U_{s,std,i}}) }{ q_{n,std,U} },\frac{U_{n,min,i}}{U_{n,std,i}} \right)
$$

$f_{U,n,max}$: 日射が当たらない外皮の部位の熱貫流率の調整係数の最大値, -  
$f_{U,n,min}$: 日射が当たらない外皮の部位の熱貫流率の調整係数の最小値, -  
$U_{A,std}$: 外皮平均熱貫流率の基準値, W/m<sup>2</sup> K  
$A_{total}$: 総外皮面積, m<sup>2</sup>  
$q_{n,std,U}$: 基準U値に基づく日射が当たらない外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$q_{s,std,U}$: 基準U値に基づく日射が当たる外皮の部位の単位温度差当たりの外皮熱損失量の合計, W/K  
$U_{s,std,i}$: 第$i$番目の日射が当たる外皮の部位の熱貫流率の基準値, W/m<sup>2</sup> K  
$U_{s,max,i}$: 第$i$番目の日射が当たる外皮の部位の熱貫流率の最大値, W/m<sup>2</sup> K  
$U_{s,min,i}$: 第$i$番目の日射が当たる外皮の部位の熱貫流率の最小値, W/m<sup>2</sup> K  
$U_{n,std,i}$: 第$i$番目の日射が当たらない外皮の部位の熱貫流率の基準値, W/m<sup>2</sup> K  
$U_{n,max,i}$: 第$i$番目の日射が当たらない外皮の部位の熱貫流率の最大値, W/m<sup>2</sup> K  
$U_{n,min,i}$: 第$i$番目の日射が当たらない外皮の部位の熱貫流率の最小値, W/m<sup>2</sup> K  

#### Function

In [24]:
def get_fUn_range(region, std_UA, total_area, qs_std_U, qn_std_U, l_sunshine_part_type, l_no_sunshine_part_type):
    
    # 日射の当たらない部位（床）の熱貫流率補正係数の最小値
    fUn_max = min(
        (std_UA * total_area - qs_std_U * max(get_min_U(x) / get_std_U(region, x) for x in l_sunshine_part_type)) / qn_std_U,
        min(get_max_U(x) / get_std_U(region, x) for x in l_no_sunshine_part_type)
    )
    
    # 日射の当たらない部位（床）の熱貫流率補正係数の最大値
    fUn_min = max(
        (std_UA * total_area - qs_std_U * min(get_max_U(x) / get_std_U(region, x) for x in l_sunshine_part_type)) / qn_std_U,
        min(get_min_U(x) / get_std_U(region, x) for x in l_no_sunshine_part_type)
    )
    
    return fUn_max, fUn_min

### 2) 熱貫流率調整係数と開口部日射熱取得率の算出

#### Outline

UA値、ηA値の基準値を満たすように、基準値計算用の住宅のU値（仕様規定の部位別U値に比例）と開口部の日射熱取得率ηH値、ηC値を決定する。  
開口部の日射熱取得率η値としては、ηH値を採用する。

$$
\displaystyle
f_{U}= \frac{ (AU_{A}) }{ (AUH) }
$$

$$
\displaystyle
η_{H}= \frac{ \frac{ (Aη_{A})_H }{100} - 0.034 \times f_{U} \times (AUν)_H }{ (Afν)_H }
$$
$$
\displaystyle
η_{C}= \frac{ \frac{ (Aη_{A})_C }{100} - 0.034 \times f_{U} \times (AUν)_C }{ (Afν)_C }
$$
$$
\displaystyle
η= η_{H}
$$

ただし、非現実的な設定は許容せず、開口部のη値は0以上0.88以下、各部位のU値は上下限値の間とする。  
そのために、調整前の冷房期と暖房期の開口部のη値が0未満または0.88超となる場合について、場合分けを行い調整を実施する。

$f_{U}$: 熱貫流率の調整係数, -  
$η$: 開口部の日射熱取得率, -  
$η_{H}$: 暖房期の開口部の日射熱取得率, -  
$η_{C}$: 冷房期の開口部の日射熱取得率, -  

$(AU_{A})$: 総外皮面積と外皮平均熱貫流率の基準値の積（後述）, W/K   
$(AUH)$: 部位の面積・長さと熱貫流率の基準値と温度差係数の積の合計（後述）, W/K 

$(Aη_{A})_H$: 総外皮面積と暖房期の外皮平均日射熱取得率の基準値の積（後述）, ㎡  
$(Aη_{A})_C$: 総外皮面積と冷房期の外皮平均日射熱取得率の基準値の積（後述）, ㎡  
$(AUν)_H$: 開口部以外の外皮の部位の面積と熱貫流率の基準値と方位係数の積の合計（後述）, W/K   
$(AUν)_C$: 開口部以外の外皮の部位の面積と熱貫流率の基準値と方位係数の積の合計（後述）, W/K   
$(Afν)_H$: 開口部の部位の面積と暖房期の日射取得率補正係数と方位係数の積の合計（後述）, W/K   
$(Afν)_C$: 開口部の部位の面積と冷房期の日射取得率補正係数と方位係数の積の合計（後述）, W/K   

#### Detail

熱貫流率の調整係数は、日射の当たる部位（屋根または天井、外壁、開口部）と、日射の当たらない部位（床）のそれぞれについて算出する。

　$f_{U,s}$ :日射の当たる部位（屋根または天井、外壁、開口部）の熱貫流率の調整係数  
　$f_{U,n}$ :日射の当たらない部位（床）の熱貫流率の調整係数

場合分けは、以下のとおりとする。  

　1 冷房期の開口部以外からの日射熱取得が多い場合（調整前$η_C<0$）  
　　1-1 暖房期の開口部以外からの日射熱取得が多い場合（調整前$η_H<0$）  
　　　　より厳しい期間に合わせて日射が当たる部位の熱貫流率を調整  
　　1-2 暖房期の日射熱取得が少ない場合（調整前$η_H>0.88$）  
　　　　UA値をもとに部位の熱貫流率を調整  
　　1-3 暖房期の日射熱取得が開口部の日射熱取得率で調整可能な範囲の場合（$0$≦調整前$η_H$≦$0.88$）  
　　　　冷房期を優先して日射が当たる部位の熱貫流率を調整  
        
　2 冷房期の日射熱取得が開口部の日射熱取得率で調整可能な範囲の場合（$0$≦調整前$η_C$≦$0.88$）  
　　2-1 暖房期の開口部以外からの日射熱取得が多い場合（調整前$η_H<0$）  
　　　　暖房期を優先して日射が当たる部位の熱貫流率を調整  
　　2-2 暖房期の日射熱取得が少ない場合（調整前$η_H>0.88$）  
　　　　暖房期を優先して日射が当たる部位の熱貫流率を調整  
　　2-3 暖房期の日射熱取得が開口部の日射熱取得率で調整可能な範囲の場合（$0$≦調整前$η_H$≦$0.88$）  
 　　　　UA値をもとに部位の熱貫流率を調整 
     
　3 冷房期の日射熱取得が少ない場合（調整前$η_C>0.88$）  
　　3-1 暖房期の開口部以外からの日射熱取得が多い場合（調整前$η_H<0$）  
　　　　UA値をもとに部位の熱貫流率を調整  
　　3-2 暖房期の日射熱取得が少ない場合（調整前$η_H>0.88$）  
　　　　より厳しい期間に合わせて日射が当たる部位の熱貫流率を調整  
　　3-3 暖房期の日射熱取得が開口部の日射熱取得率で調整可能な範囲の場合（$0$≦調整前$η_H$≦$0.88$）  
　　　　冷房期を優先して日射が当たる部位の熱貫流率を調整  

| 　 | 調整前$η_H<0$ | $0$≦調整前$η_H$≦$0.88$ | $0.88<$調整前$η_H$ |
|---|---|---|---|
| 調整前$η_C<0$ | (1-1)より厳しい期間に合わせて$f_{U,s}$を決定 | (2-1)冷房期を優先して$f_{U,s}$を決定 | (3-1)$U_A$値をもとに$f_{U,s}$を決定 | 
| 　 | $$f_{U,s} = \frac{ \frac{(Aη_A) }{100} }{ 0.034 \times (AUν) }$$ | $$f_{U,s} = \frac{ \frac{(Aη_A) }{100} }{ 0.034 \times (AUν) }$$ | $$f_{U,s} = \frac{(AU_A) }{ (AUH) }$$ |
| $0$≦調整前$η_C$≦$0.88$ | (1-2)暖房期を優先して$f_{U,s}$を決定 | (2-2)$U_A$値をもとに$f_{U,s}$を決定 | (3-2)暖房期を優先して$f_{U,s}$を決定 |
| 　 | $$f_{U,s} = \frac{ \frac{( Aη_A) }{100} }{ 0.034 \times (AUν) }$$ | $$f_{U,s} = \frac{(AU_A) }{ (AUH) }$$ | $$f_{U,s} = \frac{ \frac{( Aη_A) }{100} - 0.88 \times (Afν) }{ 0.034 \times (AUν)}$$ |
| $0.88<$調整前$η_C$ | (1-3)$U_A$値をもとに$f_{U,s}$を決定 | (2-3)冷房期を優先して$f_{U,s}$を決定 | (3-3)より厳しい期間に合わせて$f_{U,s}$を決定 |
| 　 | $$f_{U,s} = \frac{(AU_A) }{ (AUH) }$$ | $$f_{U,s} = \frac{ \frac{( Aη_A) }{100} - 0.88 \times (Afν) }{ 0.034 \times (AUν)}$$ | $$f_{U,s} = \frac{ \frac{( Aη_A) }{100} - 0.88 \times (Afν) }{ 0.034 \times (AUν)}$$ |

熱貫流率の調整係数は、下限値以上、上限値以下となるよう決定する。

$$
f_{U,s} = \left \{
\begin{array}{ll}
\displaystyle
\max \left(f_{U,s,min},\frac{ \frac{( Aη_A) }{100} }{ 0.034 \times (AUν) } \right) & \left( \frac{ \frac{(Aη_A)}{100} - 0.034 \times(AUν) }{ (Afν) } < 0.00 \right) \\
\displaystyle
\min \left(f_{U,s,max},\max \left(f_{U,s,min}, \frac{(AU_A) }{ (AUH) } \right)\right) & \left( 0.00 ≦ \frac{ \frac{(Aη_A)}{100} - 0.034 \times(AUν) }{ (Afν) } ≦ 0.88 \right) \\
\displaystyle
\min \left(f_{U,s,max},\frac{ \frac{( Aη_A) }{100} - 0.88 \times (Afν) }{ 0.034 \times (AUν)} \right) & \left( \frac{ \frac{(Aη_A)}{100} - 0.034 \times(AUν) }{ (Afν) } > 0.88 \right)
\end{array}
\right.
$$

$f_{U,s}$: 日射の当たる部位（屋根または天井、外壁、開口部）の熱貫流率を補正するための係数, -  
$f_{U,s,min}$: 日射の当たる部位（屋根または天井、外壁、開口部）の熱貫流率を補正するための係数が取り得る最小値, -  
$f_{U,s,max}$: 日射の当たる部位（屋根または天井、外壁、開口部）の熱貫流率を補正するための係数が取り得る最大値, -  

$(AU_{A})$: 総外皮面積と外皮平均熱貫流率の基準値の積（後述）, W/K   
$(Aη_{A})$: 総外皮面積と外皮平均日射熱取得率の基準値の積（後述）, ㎡  
$(AUH)$: 部位の面積と熱貫流率の基準値と温度差係数の積の合計（後述）, W/K  
$(AUν)$: 開口部以外の外皮の部位の面積と熱貫流率の基準値と方位係数の積の合計（後述）, W/K   
$(Afν)$: 開口部の部位の面積と日射取得率補正係数と方位係数の積の合計（後述）, W/K     

$$
f_{U,n} = \min \left( f_{U,n,max},\max \left( f_{U,n,min},\frac{ (AU_A) - f_{U,s} \times q_{s,std,U} }{ q_{n,std,U} } \right) \right)
$$

$f_{U,n}$: 日射の当たらない部位（床）の熱貫流率を補正するための係数, -  
$f_{U,n,min}$: 床の熱貫流率を補正するための係数が取り得る最小値, -  
$f_{U,n,max}$: 床の熱貫流率を補正するための係数が取り得る最大値, -  

開口部の日射熱取得率は、0以上、0.88以下となるよう決定する。  
開口部以外からの日射熱取得が多い場合は、日射の当たる部位の熱貫流率の調整係数を、開口部の日射熱取得率が0となるよう、熱貫流率の調整係数の下限値以上となる範囲で調整する。  
日射熱取得が少ない場合は、日射の当たる部位の熱貫流率の調整係数を、開口部の日射熱取得率が0.88となるよう、熱貫流率の調整係数の上限値以下となる範囲で調整する。

$$
η = \left \{
\begin{array}{ll}
0.00 & \left( \frac{ \frac{(Aη_A)}{100} - 0.034 \times(AUν) }{ (Afν) } < 0.00 \right) \\
\displaystyle
\min \left(0.88,\max \left(0.00, \frac{ \frac{(Aη_A)}{100} - 0.034 \times f_{U,s} \times (AUν) }{ (Afν) } \right)\right) & \left( 0.00 ≦ \frac{ \frac{(Aη_A)}{100} - 0.034 \times(AUν) }{ (Afν) } ≦ 0.88 \right) \\
0.88 & \left( \frac{ \frac{(Aη_A)}{100} - 0.034 \times(AUν) }{ (Afν) } > 0.88 \right)
\end{array}
\right.
$$

#### Function

In [27]:
def get_fU_and_eta(region, std_UA, total_area, parts, std_etaAH, std_etaAC,
                   fUs_min, fUs_max, fUn_min, fUn_max, q_std_U, qs_std_U, qn_std_U, l_sunshine_part_type):
    
    q_std = std_UA * total_area
    
    # A・ηA
    m_H_std = std_etaAH * total_area 
    m_C_std = std_etaAC * total_area 
 
    # Σ(A・U・ν)
    sum_AUnu_H = sum((part.area * get_std_U(region, part.ptype) * \
                      nu.get_nu(season=('heating' if region != 8 else 'cooling'), region=region, direction=part.direction) \
                      if (part.ptype in l_sunshine_part_type and part.ptype != 'window') else 0) \
                     for part in parts)
    sum_AUnu_C = sum((part.area * get_std_U(region, part.ptype) *  nu.get_nu(season='cooling', region=region, direction=part.direction) \
                      if (part.ptype in l_sunshine_part_type and part.ptype != 'window') else 0) \
                     for part in parts)

    # Σ(A・f・ν)   
    sum_Afnu_H = sum((part.area * f.get_f(season='heating', region=region, direction=part.direction) \
                      * nu.get_nu(season=('heating' if region != 8 else 'cooling'), region=region, direction=part.direction)) \
                     if part.ptype == 'window' else 0 for part in parts)
    sum_Afnu_C = sum((part.area * f.get_f(season='cooling', region=region, direction=part.direction) \
                      * nu.get_nu(season='cooling', region=region, direction=part.direction)) if part.ptype == 'window' else 0 
                     for part in parts)

    # 調整係数を事前に算出
    fUs_for_eta_H_000 = (m_H_std / 100 - 0.00 * sum_Afnu_H) / 0.034 / sum_AUnu_H
    fUs_for_eta_C_000 = (m_C_std / 100 - 0.00 * sum_Afnu_C) / 0.034 / sum_AUnu_C
    fUs_for_eta_H_088 = (m_H_std / 100 - 0.88 * sum_Afnu_H) / 0.034 / sum_AUnu_H
    fUs_for_eta_C_088 = (m_C_std / 100 - 0.88 * sum_Afnu_C) / 0.034 / sum_AUnu_C
    fUs_for_UA = q_std / q_std_U

    # 場合分けのためのη暫定値
    eta_H_temp = (m_H_std / 100 - 0.034 * fUs_for_UA * sum_AUnu_H) / sum_Afnu_H
    eta_C_temp = (m_C_std / 100 - 0.034 * fUs_for_UA * sum_AUnu_C) / sum_Afnu_C
    
    if eta_C_temp < 0 :
        if eta_H_temp < 0 :
            fUs = min( fUs_for_eta_H_000, fUs_for_eta_C_000 )
        elif eta_H_temp > 0.88 :
            fUs = min( fUs_max, fUs_for_UA )
        else :
            fUs = fUs_for_eta_C_000
    elif eta_C_temp > 0.88 :
        if eta_H_temp < 0 :
            fUs = min( fUs_max, fUs_for_UA )
        elif eta_H_temp > 0.88 :
            fUs = max( fUs_for_eta_H_088, fUs_for_eta_C_088)   
        else :
            fUs = fUs_for_eta_C_088
    else :
        if eta_H_temp < 0 :
            fUs = fUs_for_eta_H_000
        elif eta_H_temp > 0.88 :
            fUs = fUs_for_eta_H_088
        else :
            fUs = fUs_for_UA
    
    # 上下限値の適用
    fUs = max(fUs_min, min(fUs_max, fUs_for_UA ))
    
    # 床の熱貫流率の調整係数
    fUn = max(fUn_min, min(fUn_max, ( q_std - fUs * qs_std_U ) / qn_std_U ))
    
    # 開口部の日射熱取得率の算出
    eta = {
        'heating' : max( 0, min( 0.88, ( m_H_std / 100 - 0.034 * sum_AUnu_H * fUs ) / sum_Afnu_H)),
        'cooling' : max( 0, min( 0.88, ( m_C_std / 100 - 0.034 * sum_AUnu_C * fUs ) / sum_Afnu_C)),
          }
    eta['annual'] = eta['cooling']
   
    return fUs, fUn, eta

### 3) 基準値計算用の住宅のUA値、ηA値の検算

#### Outline

$$
\displaystyle
U_{A,check}= \frac{ \sum_{}^{}(U_i \times A_i \times H_i) + \sum_{}^{}(Ψ_i \times L_i \times H_i) }{ A_{total} }
$$

$$
\displaystyle
η_{A,H,check}= \frac{ 0.034 \times \sum_{}^{}(U_{o,i} \times A_{o,i} \times ν_{H,o,i}) + η_H \times (Afν)_H}{ A_{total} } \times 100
$$
$$
\displaystyle
η_{A,C,check}= \frac{ 0.034 \times \sum_{}^{}(U_{o,i} \times A_{o,i} \times ν_{C,o,i}) + η_C \times (Afν)_C}{ A_{total} } \times 100
$$

$U_{A,check}$: 外皮平均熱貫流率の検算値, W/(m<sup>2</sup>・K)  
$η_{A,check}$: 外皮平均日射熱取得率の検算値, -  

$(Afν)_H$: 開口部の部位の面積と暖房期の日射取得率補正係数と方位係数の積の合計（後述）, W/K   
$(Afν)_C$: 開口部の部位の面積と冷房期の日射取得率補正係数と方位係数の積の合計（後述）, W/K   

$ν$: 部位の方位係数, -  
$H$: 温度差係数, -  
$A$: 部位の面積, m<sup>2</sup>  
$L$: 部位の長さ, m  
$U$: 部位の熱貫流率, W/(m<sup>2</sup>・K)  
$Ψ$: 部位の線熱貫流率, W/(m・K)  

添字$H$: 暖房期  
添字$C$: 冷房期  
添字$i$: 外皮の第$i$番目の部位  
添字$o,i$: 開口部以外の外皮の第$i$番目の部位  
添字$w,i$: 開口部の第$i$番目の部位  

In [29]:
def check_UA_and_etaA(area_skin, l_general_parts, l_windows, l_doors, l_earthfloor_perimeters, part_U, eta, region, 
                      parts, l_sunshine_parts, l_no_sunshine_parts):
    
    # Σ(A・f・ν)    
    sum_Afnu = {
        'heating' : sum((part.area * f.get_f(season='heating', region=region, direction=part.direction) * \
                        nu.get_nu(season=('heating' if region != 8 else 'cooling'),region=region, direction=part.direction) \
                         if part.ptype == 'window' else 0) \
                        for part in parts),
        'cooling' : sum((part.area * f.get_f(season='cooling', region=region, direction=part.direction) * \
                        nu.get_nu(season='cooling', region=region, direction=part.direction) if part.ptype == 'window' else 0) \
                        for part in parts)
    }
    
    # UA値検算
    UA_check=(sum(x['area'] * part_U[x['general_part_type']] * H.get_H_by_nextspace(region, x['next_space']) 
                  for x in l_general_parts) \
              + sum(x['area'] * part_U['window'] * H.get_H_by_nextspace(region, x['next_space']) for x in l_windows) \
              + sum(x['area'] * part_U['window'] * H.get_H_by_nextspace(region, x['next_space']) for x in l_doors) \
              + sum(x['length'] * part_U['earthfloor_perimeter'] * H.get_H_by_nextspace(region, x['next_space']) 
                    for x in l_earthfloor_perimeters))/ area_skin
    
    # ηA値検算
    eta_check = {
        'heating' : (0.034 * sum(x['area'] * part_U[x['general_part_type']] * \
                                 (nu.get_nu(region=region, season=('heating' if region != 8 else 'cooling'),
                                            direction=x['direction']) if x['next_space']=='outdoor' else 0)
                                 for x in l_general_parts) \
                     + 0.034 * sum(x['area'] * part_U['door'] * \
                                 (nu.get_nu(region=region, season=('heating' if region != 8 else 'cooling'),
                                            direction=x['direction']) if x['next_space']=='outdoor' else 0)
                                 for x in l_doors) \
                     + eta['heating'] * sum_Afnu['heating']) / area_skin * 100,
        'cooling' : (0.034 * sum(x['area'] * part_U[x['general_part_type']] * \
                                 nu.get_nu(region=region, season='cooling', direction=x['direction'])
                                 for x in l_general_parts) \
                     + 0.034 * sum(x['area'] * part_U['door'] * nu.get_nu(region=region, season='cooling', direction=x['direction'])
                                 for x in l_doors) \
                     + eta['cooling'] * sum_Afnu['cooling']) / area_skin * 100,
    }
    
    return UA_check, eta_check

### 4) 基準値計算用の住宅のUA値、ηA値の調整

#### Outline

各種設定値を取得する。  
熱貫流率調整係数と開口部日射熱取得率の算出のための各種値を算出する。  
熱貫流率調整係数と開口部日射熱取得率を算出する。  
基準値計算用の住宅のUA値、ηA値の確認を行う。

熱貫流率調整係数と開口部日射熱取得率の算出のための各種値は以下のとおり。

$$
\displaystyle
(Aη_{A})_H= η_{A,H} \times A_{total} 
$$
$$
\displaystyle
(Aη_{A})_C= η_{A,C} \times A_{total} 
$$
$$
\displaystyle
(AU_{A})= U_{A,s} \times A_{total} 
$$

$U_{A,s}$: 外皮平均熱貫流率の基準値, W/(m<sup>2</sup>・K)  
$η_{A,H}$: 暖房期の外皮平均日射熱取得率の基準値, -  
$η_{A,C}$: 冷房期の外皮平均日射熱取得率の基準値, -  
$A_{total}$: 総外皮面積, m<sup>2</sup>  
添字$H$: 暖房期  
添字$C$: 冷房期  

$$
\displaystyle
(AUν)_H= \sum_{}^{}(A_{o,i} \times U_{S,o,i} \times ν_{H,o,i})
$$
$$
\displaystyle
(AUν)_C= \sum_{}^{}(A_{o,i} \times U_{S,o,i} \times ν_{C,o,i})
$$

$ν$: 部位の方位係数, -  
$A$: 部位の面積, m<sup>2</sup>  
$U_{o,s,i}$: 開口部以外の外皮の第$i$番目の部位の熱貫流率の基準値, W/(m<sup>2</sup>・K)  
添字$o,i$: 開口部以外の外皮の第$i$番目の部位  

$$
\displaystyle
(Afν)_H= \sum_{}^{}(A_{w,i} \times f_{s,w,i} \times ν_{H,w,i})
$$
$$
\displaystyle
(Afν)_C= \sum_{}^{}(A_{w,i} \times f_{s,w,i} \times ν_{C,w,i})
$$

$f$: 部位の日射取得率補正係数, -  
添字$w,i$: 開口部の第$i$番目の部位  

$U_{s}$: 開口部以外の外皮の第$i$番目の部位の熱貫流率の基準値, W/(m<sup>2</sup>・K)  
添字$i$: 外皮の第$i$番目の部位  
添字$s,i$: 日射の当たる部位（屋根または天井、外壁、開口部）の第$i$番目の部位  
添字$n,i$: 日射の当たらない部位（床）の第$i$番目の部位  

#### Function

In [ ]:
# 一般部位、窓、ドア、土間床等の外周部のリストを作成
def get_l_parts(l_general_parts, l_windows, l_doors, l_earthfloor_perimeters):

    # 一般部位
    general_parts = [Parts(area = p['area'], ptype = p['general_part_type'], nextspace = p['next_space'], direction = p['direction']) 
                     for p in l_general_parts]
    
    # 窓
    windows = [Parts(area = p['area'], ptype = 'window', nextspace = p['next_space'], direction = p['direction']) for p in l_windows]
    
    # ドア
    doors = [Parts(area = p['area'], ptype = 'door', nextspace = p['next_space'], direction = p['direction']) for p in l_doors]
    
    # 土間床等の外周部
    earthfloor_perimeters = [Parts(area = p['length'], ptype = 'earthfloor_perimeter', nextspace = p['next_space'], direction = p['direction'])
                             for p in l_earthfloor_perimeters]
    
    return general_parts + windows + doors + earthfloor_perimeters

In [ ]:
def calc_adjustment_factor( d ):
    
    house_type = d['common']['house_type']
    region     = d['common']['region']
    envelope   = d['envelope']
    total_area = get_total_area(envelope)    
    
    # 部位のリスト作成
    parts = get_l_parts(envelope['general_parts'],envelope['windows'],envelope['doors'],envelope['earthfloor_perimeters'])
    
    # 部位種類のリスト取得
    l_part_type = get_list_part_type()
    l_sunshine_part_type = get_list_sunshine_part_type()
    l_no_sunshine_part_type = get_list_no_sunshine_part_type()
    
    # 各種設定値の取得
    q_std_U = get_q_std_U(region, parts, l_part_type)
    qs_std_U = get_q_std_U(region, parts, l_sunshine_part_type)
    qn_std_U = get_q_std_U(region, parts, l_no_sunshine_part_type)
    qn_max_U = get_q_max_U(region, parts, l_no_sunshine_part_type)
    qn_min_U = get_q_min_U(region, parts, l_no_sunshine_part_type)

    # 熱貫流率の調整係数の上下限値
    fUs_max, fUs_min = get_fUs_range(region, envelope['index']['UA'], total_area, qs_std_U, qn_max_U, qn_min_U, l_sunshine_part_type)
    fUn_max, fUn_min = get_fUn_range(region, envelope['index']['UA'], total_area, qs_std_U, qn_std_U, 
                                     l_sunshine_part_type, l_no_sunshine_part_type)

    # fUs, fUn, ηの算出
    fUs, fUn, eta = get_fU_and_eta(region, envelope['index']['UA'], total_area, parts,
                                   envelope['index']['etaAH'], envelope['index']['etaAC'],
                                   fUs_min, fUs_max, fUn_min, fUn_max, q_std_U, qs_std_U, qn_std_U, l_sunshine_part_type)

    # 部位別のU値の上下限値
    part_U = { x : max(get_min_U(x), min(get_max_U(x), (fUn if x=='floor' or x =='earthfloor_perimeter' else fUs) * get_std_U(region, x))) \
              for x in ['roof','ceiling','wall','floor','window','door', 'earthfloor_perimeter'] }
    
    # 検算
    UA_check, eta_check = check_UA_and_etaA( total_area, envelope['general_parts'], envelope['windows'], envelope['doors'], \
                                            envelope['earthfloor_perimeters'], \
                                            part_U, eta, region, parts, l_sunshine_part_type, l_no_sunshine_part_type)
    
    print("U値{0},開口部η{1},開口部ηH{2},開口部ηC{3}".format(part_U, eta['annual'], eta['heating'], eta['cooling']))  
    print("UA計算値{0},ηAH計算値{1},ηAC計算値{2}".format(UA_check, eta_check['heating'], eta_check['cooling']))
    
    return eta['annual'], eta['heating'], eta['cooling'], part_U